In [1]:
pip install google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import aiplatform

In [16]:
PROJECT_ID = "629242692180"  
REGION = "asia-southeast1" 

GCS_INPUT_URI = "gs://axmt_equipment_profile/siglip_vectors"

DISPLAY_NAME = "equipment_profile"
EMBEDDING_DIMENSION = 768

In [17]:
def create_new_vector_index():
    try:
        aiplatform.init(project=PROJECT_ID, location=REGION)
        print(f"Initialized Vertex AI in {REGION}.")
        
        # 🌟 Index: metadata_schema_uri
        index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
            display_name=DISPLAY_NAME,
            contents_delta_uri=GCS_INPUT_URI,
            description="Equipment Profile",
            dimensions=EMBEDDING_DIMENSION,
            approximate_neighbors_count=5,
            leaf_node_embedding_count=1000,
            leaf_nodes_to_search_percent=7,
            index_update_method="BATCH_UPDATE",  # Options: STREAM_UPDATE, BATCH_UPDATE
            distance_measure_type=aiplatform.matching_engine.matching_engine_index_config.DistanceMeasureType.DOT_PRODUCT_DISTANCE
        )
        
        return index

    except Exception as e:
        print(f"!!! An error occurred during Index creation: {e}")
        return None

In [18]:
create_new_vector_index()

Initialized Vertex AI in asia-southeast1.
Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/629242692180/locations/asia-southeast1/indexes/2127801290351181824/operations/1420560501871476736
MatchingEngineIndex created. Resource name: projects/629242692180/locations/asia-southeast1/indexes/2127801290351181824
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/629242692180/locations/asia-southeast1/indexes/2127801290351181824')


resource name: projects/629242692180/locations/asia-southeast1/indexes/2127801290351181824

In [ ]:
my_index = "2302878725865209856"
my_deployed_index = "equipment_profile"
my_index_endpoint.deploy_index(
    index=my_index,
    deployed_index_id=my_deployed_index,
    display_name=DISPLAY_NAME,
    machine_type="n1-standard-2", # Or other machine type
    min_replica_count=1,
    max_replica_count=2
)

In [8]:
INDEX_RESOURCE_NAME = "projects/629242692180/locations/asia-southeast1/indexes/1120120873727033344" 

aiplatform.init(project=PROJECT_ID, location=REGION)

index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)
    

In [12]:
IS_COMPLETE_OVERWRITE = False
update_operation = index.update_embeddings(
    contents_delta_uri=GCS_INPUT_URI, is_complete_overwrite=IS_COMPLETE_OVERWRITE
)

print(f"Update operation started: {update_operation.operation.name}")
print("This operation will process all files in the GCS path (including new ones).")

update_operation.wait_for_resource_creation()

print("✅ Index contents update completed.")

Updating MatchingEngineIndex index: projects/629242692180/locations/asia-southeast1/indexes/1120120873727033344
Update MatchingEngineIndex index backing LRO: projects/629242692180/locations/asia-southeast1/indexes/1120120873727033344/operations/8074628951311384576


KeyboardInterrupt: 